# Simple Backtest Example

This notebook demonstrates a fast backtest of portfolio optimization algorithms during a volatile period where A2A ensemble methods can shine.

## 1. Import Libraries

Import the necessary libraries for backtesting.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

# AlloOptim imports
from allooptim.backtest.backtest_config import BacktestConfig
from allooptim.backtest.backtest_engine import BacktestEngine

print("Libraries imported successfully!")

: 

## 2. Configure Backtest

Set up a fast backtest for a volatile period (COVID crash) where A2A ensemble methods excel.

In [ ]:
# Configure a fast backtest for volatile period (COVID crash)
config = BacktestConfig(
    # Short volatile period where A2A shines (March-June 2020)
    quick_start_date=datetime(2020, 3, 1),
    quick_end_date=datetime(2020, 6, 30),
    # Fast settings for quick execution
    rebalance_frequency=5,  # Rebalance every 5 trading days
    lookback_days=30,  # Use 30 days of historical data
    # Simple asset universe
    symbols=["AAPL", "MSFT", "AMZN", "GOOGL", "TSLA", "SPY"],
    # Use A2A with a few diverse optimizers
    optimizer_names=["MaxSharpe", "RiskParityOptimizer", "NaiveOptimizer", "MomentumOptimizer", "HRPOptimizer", "NCOSharpeOptimizer"],
    # Quick test mode
    quick_test=True,
)

## 3. Run Backtest

Execute the fast backtest to see how individual optimizers and A2A ensemble perform during market volatility.

In [ ]:
# Initialize and run the backtest
print("Initializing backtest engine...")
backtest_engine = BacktestEngine(config)

print("Running fast backtest...")
print("This should complete in under 30 seconds...")

# Run the backtest
results = backtest_engine.run_backtest()

if results:
    print("✅ Backtest completed successfully!")
    print(f"   Tested {len([k for k in results.keys() if k != 'SPY_Benchmark'])} optimizers + A2A + benchmark")
else:
    print("❌ Backtest failed")
    raise RuntimeError("Backtest execution failed")

## 4. Analyze Results

Compare the performance of individual optimizers vs A2A ensemble during the volatile COVID period.

In [ ]:
# Extract performance data for analysis
performance_data = []

for name, data in results.items():
    if 'metrics' in data and 'sharpe_ratio' in data['metrics']:
        row = {'strategy': name}
        row.update(data['metrics'])
        performance_data.append(row)

# Create DataFrame for analysis
df_results = pd.DataFrame(performance_data)
df_results = df_results.sort_values('sharpe_ratio', ascending=False)

print("Backtest Results Summary (COVID Crash Period):")
print("=" * 60)
print(f"Test period: March-June 2020 (High volatility)")
print(f"Strategies tested: {len(df_results)}")
print()

# Show top performers
print("Performance Rankings:")
print("-" * 40)
for i, row in df_results.head().iterrows():
    print("2d")

print()

# Check if A2A outperformed individual optimizers
a2a_sharpe = df_results[df_results['strategy'] == 'A2A_Ensemble']['sharpe_ratio'].iloc[0]
individual_avg = df_results[df_results['strategy'] != 'A2A_Ensemble']['sharpe_ratio'].mean()
spy_sharpe = df_results[df_results['strategy'] == 'SPY_Benchmark']['sharpe_ratio'].iloc[0]

print("A2A Ensemble Analysis:")
print("-" * 40)
print(".3f")
print(".3f")
print(".3f")
print(".3f")
print()

# Create visualization
plt.figure(figsize=(15, 10))

# Sharpe ratio comparison
plt.subplot(2, 2, 1)
strategies = df_results['strategy']
sharpe_ratios = df_results['sharpe_ratio']
bars = plt.bar(range(len(strategies)), sharpe_ratios, color='skyblue')
plt.xticks(range(len(strategies)), strategies, rotation=45, ha='right')
plt.title('Sharpe Ratio Comparison')
plt.ylabel('Sharpe Ratio')
plt.grid(axis='y', alpha=0.3)

# Highlight A2A
a2a_idx = strategies[strategies == 'A2A_Ensemble'].index[0]
bars[a2a_idx].set_color('green')
bars[a2a_idx].set_label('A2A Ensemble')

plt.legend()

# Portfolio value over time (normalized)
plt.subplot(2, 2, 2)
for name, data in results.items():
    if 'portfolio_values' in data:
        portfolio_values = data['portfolio_values']
        if not portfolio_values.empty:
            # Normalize to start at 100
            normalized = (portfolio_values / portfolio_values.iloc[0]) * 100
            plt.plot(normalized.index, normalized.values, label=name, linewidth=2)

plt.title('Portfolio Value Over Time (COVID Crash)')
plt.xlabel('Date')
plt.ylabel('Portfolio Value ($)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(alpha=0.3)

# Maximum drawdown comparison
plt.subplot(2, 2, 3)
max_drawdowns = df_results['max_drawdown']
bars = plt.bar(range(len(strategies)), max_drawdowns, color='salmon')
plt.xticks(range(len(strategies)), strategies, rotation=45, ha='right')
plt.title('Maximum Drawdown Comparison')
plt.ylabel('Max Drawdown')
plt.grid(axis='y', alpha=0.3)

# Highlight A2A
bars[a2a_idx].set_color('orange')

# Annual return vs volatility
plt.subplot(2, 2, 4)
annual_returns = df_results['annual_return']
annual_vols = df_results['annual_volatility']

plt.scatter(annual_vols, annual_returns, s=100, alpha=0.7)

# Add labels for each point
for i, row in df_results.iterrows():
    plt.annotate(row['strategy'],
                (row['annual_volatility'], row['annual_return']),
                xytext=(5, 5), textcoords='offset points',
                fontsize=8, alpha=0.8)

plt.title('Risk-Return Profile')
plt.xlabel('Annual Volatility')
plt.ylabel('Annual Return')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Key Insights:")
print("-" * 40)
print("• A2A ensemble methods often perform better during high volatility")
print("• Individual optimizers can be sensitive to parameter choices")
print("• Ensemble approaches provide more robust risk-adjusted returns")
print("• COVID period (March-June 2020) was ideal for testing A2A benefits")